In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input-28x28 | output-28x28 | RF-3x3
        self.bn1 = nn.BatchNorm2d(8) 
        self.do1 = nn.Dropout2d(0.1)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #input-28x28 | output-28x28 | RF-5x5
        self.bn2 = nn.BatchNorm2d(16)
        self.do2 = nn.Dropout2d(0.1)

        self.pool1 = nn.MaxPool2d(2, 2) #input-28x28 | output-14x14 | RF-10x10

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #input-14x14 | output-14x14 | RF-12x12
        self.bn3 = nn.BatchNorm2d(32)
        self.do3 = nn.Dropout2d(0.1)

        self.conv4 = nn.Conv2d(32, 8, 1) #input-14x14 | output-14x14 | RF-14x14
        #self.bn4 = nn.BatchNorm2d(8)
        #self.do4 = nn.Dropout2d(0.1)

        self.pool2 = nn.MaxPool2d(2, 2) #7  #input-28x28 | output-28x28 | RF-3x3

        self.conv5 = nn.Conv2d(8, 16, 3, padding=1) #7  #input-28x28 | output-28x28 | RF-3x3
        self.bn5 = nn.BatchNorm2d(16)
        self.do5 = nn.Dropout2d(0.1)


        self.conv6 = nn.Conv2d(16,32,3, padding=1) #7  #input-28x28 | output-28x28 | RF-3x3
        self.bn6 = nn.BatchNorm2d(32)
        self.do6 = nn.Dropout2d(0.1)

        self.pool3 = nn.MaxPool2d(2, 2) #3  #input-28x28 | output-28x28 | RF-3x3

        self.conv7 = nn.Conv2d(32,8,1) #3  #input-28x28 | output-28x28 | RF-3x3
        #self.bn7 = nn.BatchNorm2d(64)
        #self.do7 = nn.Dropout2d(0.1)


        self.conv8 = nn.Conv2d(8,16,3, padding=1) #3  #input-28x28 | output-28x28 | RF-3x3
        self.bn8 = nn.BatchNorm2d(16)
        self.do8 = nn.Dropout2d(0.1)

        self.conv9 = nn.Conv2d(16,32,3, padding=1) #3  #input-28x28 | output-28x28 | RF-3x3
        self.bn9 = nn.BatchNorm2d(32)
        self.do9 = nn.Dropout2d(0.1)


        self.aap2d = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(32*1*1,10)
        self.fc2 = nn.Linear(32,10)


    def forward(self, x):
        x = self.pool1(self.do2(self.bn2(F.relu(self.conv2(self.do1(self.bn1(F.relu(self.conv1(x)))))))))
        x = self.pool2(F.relu(self.conv4(self.do3(self.bn3(F.relu(self.conv3(x)))))))
        x = self.do6(self.bn6(F.relu(self.conv6(self.do5(self.bn5(F.relu(self.conv5(x))))))))
        x = self.conv7(x)
        x = self.do9(self.bn9(self.conv9(self.do8(self.bn8(self.conv8(x))))))
        x = self.aap2d(x)
        #print(x.shape)
        x = x.reshape(-1, 32*1*1)
        x = self.fc1(x)
        #x = self.fc2(x)
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
print(model)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do1): Dropout2d(p=0.1, inplace=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do2): Dropout2d(p=0.1, inplace=False)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do3): Dropout2d(p=0.1, inplace=False)
  (conv4): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(8, 16, kernel_size=(3, 3), st

<ipython-input-10-24526b341a74>:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.007, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-10-24526b341a74>:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.25160324573516846 batch_id=468: 100%|██████████| 469/469 [01:25<00:00,  5.47it/s]



Test set: Average loss: 0.1197, Accuracy: 9660/10000 (96.60%)



loss=0.09150660783052444 batch_id=468: 100%|██████████| 469/469 [01:22<00:00,  5.70it/s]



Test set: Average loss: 0.0687, Accuracy: 9811/10000 (98.11%)



loss=0.08957917243242264 batch_id=468: 100%|██████████| 469/469 [01:23<00:00,  5.65it/s]


...


Test set: Average loss: 0.0250, Accuracy: 9915/10000 (99.15%)



loss=0.03384203836321831 batch_id=468: 100%|██████████| 469/469 [01:23<00:00,  5.64it/s]



Test set: Average loss: 0.0272, Accuracy: 9907/10000 (99.07%)



loss=0.09015195816755295 batch_id=468: 100%|██████████| 469/469 [01:23<00:00,  5.59it/s]



Test set: Average loss: 0.0234, Accuracy: 9926/10000 (99.26%)

